_________________________
_________________________

Setup and Requirements

_________________________
_________________________

In [ ]:
# This is your directory path to Salicon images
path_to_images = "...Your path/val/Images_val"

# This is your directory path to ground truth saliency maps
path_to_maps = "...Your path/val/Maps_val"

# This is where you will save results to
save_path = "...Your path/val/backup.npy"

# Turn recovery mode to True if you want to use a backup 
recovery_mode = False


In [ ]:
# Install dependencies (this is what I used)
!pip install numpy==1.26.4 \
             opencv-python==4.9.0.80 \
             matplotlib==3.8.3 \
             Pillow==10.2.0 \
             scipy==1.12.0 \
             scikit-learn==1.5.2 \
             torch==2.8.0 \
             torchvision==0.23.0 \
             grad-cam==1.5.5

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import gc
import os
from PIL import Image
from sklearn.metrics import roc_auc_score
from scipy.ndimage import gaussian_filter
from pytorch_grad_cam import EigenCAM
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from pytorch_grad_cam.utils.reshape_transforms import fasterrcnn_reshape_transform
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
class MultiEigenCAM(EigenCAM):
    def __init__(self, model, target_layers, target_index=0, layer='pool', reshape_transform=None):
        # We explicitly pass the required arguments to the parent class
        super(MultiEigenCAM, self).__init__(model, 
                                            target_layers, 
                                            reshape_transform=self.modified_fasterrcnn_reshape_transform)
        self.target_index = target_index
        self.S = None
        self.rf_weights = None
        self.layer = layer

    def get_cam_image(self, 
                      input_tensor,
                      target_layer,
                      targets,
                      activations,
                      grads,
                      eigen_smooth):
        '''
        Override inherited method, almost idenital except stores SVD results, 
        changes projection to specific target_index instead of top 1 PC 
        '''
        
        # TBD: use pytorch batch svd implementation
        activations[np.isnan(activations)] = 0
        projections = []

        for a in activations:
            reshaped_activations = (a).reshape(
                a.shape[0], -1).transpose()
            # Centering before the SVD seems to be important here,
            # Otherwise the image returned is negative
            reshaped_activations = reshaped_activations - \
                reshaped_activations.mean(axis=0)
            
            # Perform SVD
            U, S, VT = np.linalg.svd(reshaped_activations, full_matrices=True)
            
            self.S = S
            self.rf_weights = VT

            projection = reshaped_activations @ VT[self.target_index, :]
            projection = projection.reshape(a.shape[1:])
            projections.append(projection)

        return np.float32(projections)
    
    def calculate_explained_variance(self, numComponents = 1):
        '''
        Find total % of variance explained by top numComponents 
        '''

        total_explained = 0
        squared_s = self.S ** 2

        for i in range(0,numComponents):
            
            # 2. Calculate the ratio for the specific target_index
            # Formula: component_variance / total_variance
            variance_ratio = squared_s[i] / np.sum(squared_s)
            total_explained += variance_ratio

        return total_explained
    
    def modified_fasterrcnn_reshape_transform(self, x):
        '''
        Override inherited method, everything is the same except
        target_size reshapes into specified layer size instead of only "pool"
        '''

        target_size = x[self.layer].size()[-2:]
        activations = []
        for key, value in x.items():
            activations.append(
                torch.nn.functional.interpolate(
                    torch.abs(value),
                    target_size,
                    mode='bilinear'))
        activations = torch.cat(activations, axis=1)
        return activations



In [ ]:
class SaliencyMetrics:
    """
    Standard saliency metrics based on MIT Saliency Benchmark
    ALL METHODS HERE ARE STATIC, meaning this class is just for ease of namespace
    """
    
    @staticmethod
    def normalize(smap):
        """Normalize to [0, 1]"""
        smap = smap - smap.min()
        if smap.max() > 0:
            smap = smap / smap.max()
        return smap
    
    @staticmethod
    def cc(smap1, smap2):
        """
        Pearson's Correlation Coefficient (CC)
        Range: [-1, 1], higher is better
        """
        smap1 = SaliencyMetrics.normalize(smap1).flatten()
        smap2 = SaliencyMetrics.normalize(smap2).flatten()
        
        # Remove mean
        smap1 = smap1 - smap1.mean()
        smap2 = smap2 - smap2.mean()
        
        return np.corrcoef(smap1, smap2)[0, 1]

    @staticmethod
    def cc_mask(smap1, smap2, threshold=1e-4):
        """
        Pearson's Correlation Coefficient (CC)
        Range: [-1, 1], higher is better
        
        Args:
            threshold: Values below this are considered 'background/zero'.
                       Only pixels active in at least one map are compared.
        """
        # 1. Normalize first so thresholding is consistent (0 to 1)
        smap1 = SaliencyMetrics.normalize(smap1).flatten()
        smap2 = SaliencyMetrics.normalize(smap2).flatten()
        
        # 2. Create the Union Mask
        # We want to keep pixels where EITHER map 1 OR map 2 is active.
        # This removes the "common zeros" that inflate correlation.
        active_mask = (smap1 > threshold) | (smap2 > threshold)
        
        # 3. Apply the mask
        smap1_active = smap1[active_mask]
        smap2_active = smap2[active_mask]
        
        # 4. Handle Edge Case: If maps are empty (all zeros), return 0
        # We need at least 2 points to calculate variance/correlation
        if len(smap1_active) < 2:
            return 0.0
        
        # 5. Compute Correlation
        return np.corrcoef(smap1_active, smap2_active)[0, 1]
    
    @staticmethod

    def nss(smap, fixation_map):
        """
        Normalized Scanpath Saliency (NSS)
        Range: [-∞, ∞], higher is better, >1 is good
        
        Args:
            smap: predicted saliency map
            fixation_map: binary or continuous fixation map
        """
        smap_normalized = (smap - smap.mean()) / (smap.std() + 1e-8)
        fixation_points = fixation_map > 0
        
        if not fixation_points.any():
            return 0.0
        
        return smap_normalized[fixation_points].mean()
    
    # def nss(smap, fixation_map):
    #     """
    #     Normalized Scanpath Saliency (NSS)
    #     Range: [-∞, ∞], higher is better, >1 is good
        
    #     Args:
    #         smap: predicted saliency map
    #         fixation_map: binary or continuous fixation map
    #     """
    #     # Normalize saliency map to have zero mean and unit std
    #     smap_normalized = (smap - smap.mean()) / (smap.std() + 1e-8)
        
    #     # Get fixation locations
    #     if fixation_map.max() > 0:
    #         fixation_map = fixation_map / fixation_map.max()
        
    #     # Compute NSS
    #     nss_value = (smap_normalized * fixation_map).sum() / (fixation_map.sum() + 1e-8)
        
    #     return nss_value
    
    @staticmethod
    def auc_judd(smap, fixation_map, num_splits=100):
        """
        AUC Judd (Area Under ROC Curve)
        Range: [0, 1], higher is better, 0.5 is chance
        
        Treats fixated pixels as positive, all others as negative
        """
        smap_flat = smap.flatten()
        fixation_flat = (fixation_map > 0).astype(int).flatten()
        
        # Need both positive and negative samples
        if len(np.unique(fixation_flat)) < 2:
            return np.nan
        
        try:
            return roc_auc_score(fixation_flat, smap_flat)
        except:
            return np.nan
    
    @staticmethod
    def auc_shuffled(smap, fixation_map, other_maps=None, num_splits=100):
        """
        AUC Shuffled (using other images as negative samples)
        More realistic than AUC Judd
        """
        # Simplified version - use threshold-based approach
        smap_flat = smap.flatten()
        
        # Positive samples: actual fixations
        pos_indices = fixation_map.flatten() > 0
        
        if pos_indices.sum() == 0:
            return np.nan
        
        # Negative samples: non-fixated locations
        neg_indices = ~pos_indices
        
        if neg_indices.sum() == 0:
            return np.nan
        
        # Create binary labels
        y_true = np.concatenate([np.ones(pos_indices.sum()), 
                                 np.zeros(min(neg_indices.sum(), pos_indices.sum() * 10))])
        y_scores = np.concatenate([smap_flat[pos_indices],
                                   np.random.choice(smap_flat[neg_indices], 
                                                   min(neg_indices.sum(), pos_indices.sum() * 10))])
        
        try:
            return roc_auc_score(y_true, y_scores)
        except:
            return np.nan
    
    @staticmethod
    def kld(smap, fixation_map):
        """
        Kullback-Leibler Divergence (KLD)
        Range: [0, ∞], lower is better
        """
        # Convert to probability distributions
        smap = SaliencyMetrics.normalize(smap)
        fixation_map = SaliencyMetrics.normalize(fixation_map)
        
        smap_flat = smap.flatten()
        fix_flat = fixation_map.flatten()
        
        # Convert to distributions
        eps = 1e-8
        smap_prob = smap_flat / (smap_flat.sum() + eps)
        fix_prob = fix_flat / (fix_flat.sum() + eps)
        
        # Avoid log(0)
        smap_prob = np.maximum(smap_prob, eps)
        fix_prob = np.maximum(fix_prob, eps)
        
        return np.sum(fix_prob * np.log(fix_prob / smap_prob))
    
    @staticmethod
    def compute_all_metrics(predicted, ground_truth):
        """
        Compute all metrics at once
        
        Args:
            predicted: predicted saliency map (H, W)
            ground_truth: ground truth saliency/fixation map (H, W)
        
        Returns:
            dict of metrics
        """
        # Ensure same size
        if predicted.shape != ground_truth.shape:
            predicted = cv2.resize(predicted, 
                                  (ground_truth.shape[1], ground_truth.shape[0]))
        
        metrics = {}
        

        try:
            metrics['CC_mask'] = SaliencyMetrics.cc_mask(predicted, ground_truth)
        except:
            metrics['CC_mask'] = np.nan
        
        try:
            metrics['CC_standard'] = SaliencyMetrics.cc(predicted, ground_truth)
        except:
            metrics['CC_standard'] = np.nan
        
        try:
            metrics['NSS'] = SaliencyMetrics.nss(predicted, ground_truth)
        except:
            metrics['NSS'] = np.nan
        
        try:
            metrics['AUC-Judd'] = SaliencyMetrics.auc_judd(predicted, ground_truth)
        except:
            metrics['AUC-Judd'] = np.nan
        
        try:
            metrics['KLD'] = SaliencyMetrics.kld(predicted, ground_truth)
        except:
            metrics['KLD'] = np.nan
        
        # Additional simple metrics
        pred_norm = SaliencyMetrics.normalize(predicted)
        gt_norm = SaliencyMetrics.normalize(ground_truth)
        
        metrics['MAE'] = np.mean(np.abs(pred_norm - gt_norm))
        metrics['MSE'] = np.mean((pred_norm - gt_norm) ** 2)
        
        return metrics


_________________________
_________________________

Main Loop

_________________________
_________________________

In [ ]:
def generate_cams(image_loc, saliency_path, title, orig_cor = -100.00, CAMS=[], output_saliency_metrics = False, output_display = False):
    
    image = np.array(Image.open(image_loc))
    image_float_np = np.float32(image) / 255

    # Transform image into a tensor
    
    # Construct Model
    # Define the torchvision image transforms
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])
    input_tensor = transform(image)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_tensor = input_tensor.to(device)

    # Add a batch dimension:
    input_tensor = input_tensor.unsqueeze(0)

    # Declare model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # All FPN layers
    target_layers = [model.backbone]

    # Make a custom eigenCAM
    cam = MultiEigenCAM(model,
                target_layers,
                target_index=0,
                reshape_transform=fasterrcnn_reshape_transform)

    # Not actually used for EigenCAM, only for gradient based
    targets = [FasterRCNNBoxScoreTarget(labels=[], bounding_boxes=[])]

    # Produce saliency map
    with torch.no_grad():  # Avoid unnecessary gradient computation
        grayscale_cam = cam(input_tensor, targets=targets)

    # Take the first image in the batch (there's just only one in this implementation)
    grayscale_cam = grayscale_cam[0, :] # Take all rows and columns

    # Apply "Fairness" Blur (sigma = 19)
    # This converts the "Blocky/Grid" activation into a "Probability" distribution matching the SALICON style.
    smooth_cam = gaussian_filter(grayscale_cam, sigma=19)

    # Normalize again (optional but recommended for visualization/metrics)
    smooth_cam = smooth_cam - np.min(smooth_cam)
    smooth_cam = smooth_cam / (np.max(smooth_cam) + 1e-7)

    # Load the ground truth saliency map (PNG), Normalize to [0, 1]
    gt_saliency_np = np.array(Image.open(saliency_path)).astype(np.float32) / 255.0

    # Append to CAMS if output_saliency_metrics = True
    if output_saliency_metrics == True:

        CAMS.append(SaliencyMetrics.compute_all_metrics(grayscale_cam, gt_saliency_np))
        CAMS.append(SaliencyMetrics.compute_all_metrics(smooth_cam, gt_saliency_np))

    if output_display == True:

        # Overlay saliency map on original image
        cam_image = show_cam_on_image(image_float_np, grayscale_cam, use_rgb=True)
        smooth_cam_image = show_cam_on_image(image_float_np, smooth_cam, use_rgb=True)
        gt_map = show_cam_on_image(image_float_np, gt_saliency_np, use_rgb=True)

        fig, axes = plt.subplots(1, 4, figsize=(18, 6))

        fig.suptitle(f"Heatmaps for Image ID: {title} \n Correlation: {orig_cor:.02}", fontsize=20)

        axes[0].imshow(image_float_np)
        axes[0].set_title('Original Image', fontsize=14)
        axes[0].axis('off')

        axes[1].imshow(cam_image)
        axes[1].set_title('FastRCNN + CAM', fontsize=14)
        axes[1].axis('off')

        axes[2].imshow(smooth_cam_image)
        axes[2].set_title('SMOOTHED CAM', fontsize=14)
        axes[2].axis('off')

        axes[3].imshow(gt_map)
        axes[3].set_title('Ground Truth', fontsize=14)
        axes[3].axis('off')

        plt.tight_layout()
        plt.show()
        
    # Delete cam
    del cam   
    gc.collect()


In [ ]:
def display_images(main_dict, keys=[]):
    '''
    Pass this the results obtained, display output graphs row by row
    '''

    for key in keys:
    
        # Extract info from keys
        generate_cams(main_dict[key[0]][0], main_dict[key[0]][1], title=key[0][-12:], orig_cor=key[1], output_display=True)
     

In [ ]:
# Main Loop (skip if recovery mode)

if recovery_mode == False:

    # Dictionary of path pairs, image + ground truth saliency map, as well as all outputs of processing
    main_dict = {}

    # Loop through source image list
    for filename in os.listdir(path_to_images): 
        
        # Make a dict key name if doesn't exist
        dict_key = filename[:-4]
        if (dict_key in main_dict) == False:
            main_dict[dict_key] = [os.path.join(path_to_images, filename)]

    # Loop through ground truth list
    for filename in os.listdir(path_to_maps):

        # Check corresponding source image
        dict_key = filename[:-4]
        if (dict_key in main_dict):
            main_dict[dict_key].append(os.path.join(path_to_maps, filename))

    # This should be 5000 images if done properly
    print(f"Total # of images: {len(main_dict)}")

    # Main Loop
    count = 0
    keys = []

    for key in main_dict:
        
        image_loc = main_dict[key][0]
        saliency_path = main_dict[key][1]
        CAMS = [] # Empty cams
        title = key[-12:]

        # Returns a list of source image + CAMS 
        # By default, CAM objects will have Principal Component Metrics already
        # Set output_saliency_metrics=True to get metrics when comparing to ground truth 
        # Set output_display=True to display side by side heatmaps w/ original image
        generate_cams(image_loc, saliency_path, title, CAMS=CAMS, output_saliency_metrics=True, output_display=False)
        main_dict[key].append(CAMS)
        keys.append([key, main_dict[key][-1][-1]["CC_mask"], main_dict[key][-1][-1]["CC_standard"], main_dict[key][-1][-1]["NSS"]])
        

In [ ]:
# Make a backup file (VERY IMPORTANT, otherwise you'll run another 6+ hours)

if recovery_mode == False:
    
    os.makedirs(os.path.dirname(save_path), exist_ok = True)  # Create dirs if needed
    np.save(save_path, main_dict, allow_pickle=True)

else:
    
    # To load it back:
    main_dict = np.load(save_path, allow_pickle=True).item()
    keys = []
    for key in main_dict:
        keys.append([key, main_dict[key][-1][-1]["CC_mask"], main_dict[key][-1][-1]["CC_standard"], main_dict[key][-1][-1]["NSS"]])


In [ ]:
# Display top, middle, and bottom n images (default n = 25, change is needed)
n = 25

# Display most correlated images
top_sorted_keys = sorted(keys, key=lambda x: x[2], reverse=True)
display_images(main_dict, top_sorted_keys[0:n])


In [ ]:
# Display images with near 0 correlation

# Display least correlated images
bot_sorted_keys = sorted(keys, key=lambda x: x[1])
display_images(main_dict, bot_sorted_keys[2500:2500+n])

In [ ]:
# Display least correlated images
bot_sorted_keys = sorted(keys, key=lambda x: x[2])
display_images(main_dict, bot_sorted_keys[0:n])


In [ ]:
# Summary metrics across all 5000 images

cc_values = [k[1] for k in keys]

# Create decile bins
deciles = np.percentile(cc_values, np.arange(0, 101, 10))

# Assign each value to a decile and compute averages
decile_averages = []
decile_counts = []

for i in range(10):
    # Get values in this decile range
    lower = deciles[i]
    upper = deciles[i + 1]
    
    # Include upper bound for last decile
    if i == 9:
        bucket_values = [v for v in cc_values if lower <= v <= upper]
    else:
        bucket_values = [v for v in cc_values if lower <= v < upper]
    
    decile_averages.append(np.mean(bucket_values) if bucket_values else 0)
    decile_counts.append(len(bucket_values))

# Step 4: Display results
print("Decile Analysis of CC_mask:")
print("-" * 60)
for i in range(10):
    print(f"Decile {i+1:2d} [{deciles[i]:6.3f} - {deciles[i+1]:6.3f}]: "
          f"avg = {decile_averages[i]:.4f}, count = {decile_counts[i]:4d}"
          f" ")
print("-" * 60)
print(f"Overall average: {np.mean(cc_values):.4f}")


_________________________
_________________________

Deeper Dive into Results

_________________________
_________________________

In [ ]:
def deep_dive(image_loc, saliency_path, title, orig_cor, CAMS=[], invert=False):

    image = np.array(Image.open(image_loc))
    image_float_np = np.float32(image) / 255

    # Transform image into a tensor

    # Construct Model
    # Define the torchvision image transforms
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ])

    input_tensor = transform(image)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_tensor = input_tensor.to(device)

    # Add a batch dimension:
    input_tensor = input_tensor.unsqueeze(0)

    # Declare model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    target_layers = [model.backbone]

    # Load the ground truth saliency map (PNG), Normalize to [0, 1]
    gt_saliency_np = np.array(Image.open(saliency_path)).astype(np.float32) / 255.0

    gt_map = show_cam_on_image(image_float_np, gt_saliency_np, use_rgb=True)

    # Make EigenCAM maps for 3 spatial dimensions
    cor_after = -100.00 # This is the max of FPN layer 2 and layer 3 dimensions
    layer_after = 0

    for i in range (2,5):

        if i != 4:
            cam = MultiEigenCAM(model,
                        target_layers,
                        layer=str(i),
                        target_index=0,
                        reshape_transform=fasterrcnn_reshape_transform)
            
        else:
            cam = MultiEigenCAM(model,
                        target_layers,
                        layer='pool',
                        target_index=0,
                        reshape_transform=fasterrcnn_reshape_transform)
    
        # Not actually used for EigenCAM, only for gradient based
        targets = [FasterRCNNBoxScoreTarget(labels=[], bounding_boxes=[])]

        # Produce saliency map
        with torch.no_grad():  # Avoid unnecessary gradient computation
            grayscale_cam = cam(input_tensor, targets=targets)

        # Take the first image in the batch (there's just only one in this implementation)
        grayscale_cam = grayscale_cam[0, :] # Take all rows and columns

        # Do an experimental flip of signs to get "correct" signs
        if i == 4 and invert == True:
            grayscale_cam = -grayscale_cam

        # Apply "Fairness" Blur (sigma = 19)
        # This converts the "Blocky/Grid" activation into a "Probability" distribution matching the SALICON style.
        smooth_cam = gaussian_filter(grayscale_cam, sigma=19)

        # Normalize again (optional but recommended for visualization/metrics)
        smooth_cam = smooth_cam - np.min(smooth_cam)
        smooth_cam = smooth_cam / (np.max(smooth_cam) + 1e-7)

        # Overlay saliency map on original image
        smooth_cam_image = show_cam_on_image(image_float_np, smooth_cam, use_rgb=True)
        CAMS.append(smooth_cam_image)

        # Check which correlation is larger, replace if needed

        cc = SaliencyMetrics.cc_mask(smooth_cam, gt_saliency_np)
        if cor_after < cc:
            cor_after = cc
            layer_after = str(i) 

    if layer_after == '2':
        layer_after = f"(FPN layer {layer_after}, 30x40)"
    elif layer_after == '3':
        layer_after = f"(FPN layer {layer_after}, 15x20)"
    else:
        layer_after = f"(FPN pool layer, 8x10)"

    fig, axes = plt.subplots(1, 5, figsize=(18, 6))

    fig.suptitle(f"Spatial Comparison Image ID: {title} \n Default Cor: {orig_cor:.02} ------ Cor After: {cor_after:.02} {layer_after}", fontsize=20)

    axes[0].imshow(image_float_np)
    axes[0].set_title('Original Image', fontsize=14)
    axes[0].axis('off')

    axes[1].imshow(CAMS[-1])
    axes[1].set_title('8x10 (Pool Flipped)', fontsize=14)
    axes[1].axis('off')

    axes[2].imshow(CAMS[-2])
    axes[2].set_title('15x20 (Blurring)', fontsize=14)
    axes[2].axis('off')

    axes[3].imshow(CAMS[-3])
    axes[3].set_title('30x40 (Normal)', fontsize=14)
    axes[3].axis('off')

    axes[4].imshow(gt_map)
    axes[4].set_title('Ground Truth', fontsize=14)
    axes[4].axis('off')

    plt.tight_layout()
    plt.show()
        
    # Delete cam
    del cam   
    gc.collect()
    

In [ ]:
# Do a deeper dive into bottom n by changing spatial dimensions

# Top n images to look at
n = 25

# Display least correlated images

for i in range(0, n):
    
    bot_sorted_keys = sorted(keys, key=lambda x: x[2])
    image_loc = main_dict[bot_sorted_keys[i][0]][0]
    saliency_path = main_dict[bot_sorted_keys[i][0]][1]
    title = bot_sorted_keys[i][0][-12:]
    orig_cor = main_dict[bot_sorted_keys[i][0]][-1][-1]['CC_mask']

    deep_dive(image_loc, saliency_path, title, orig_cor, invert=True)
